In [1]:
from packages import Marksheet

marker_positions = [
    ( 2, 26),
    ( 2, 28),
    ( 2, 56),
    ( 3, 24),
    (40,  2),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/back_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions = marker_positions,
                  monitor= True)

init


In [2]:
from packages import Marksheet

marker_positions = [
    ( 2, 28),
    ( 2, 56),
    (40,  2),
    (40,  4),
    (40, 56),
]

sheet = Marksheet(path = "./files/scan/top_sample.jpg", 
                  row = 41, column = 57, 
                  marker_positions=marker_positions,
                  monitor= True)

init


In [3]:
sheet.read()

open ./files/scan/top_sample.jpg
read image shape: (2336, 1647, 3)


In [4]:
sheet.rotation()
sheet.aliment()

rotated to horizontal
make reference image shape: (1566, 2254) reqtangle h,w: 40.170731707317074 40.98245614035088
rotation: 0, max_val: 0.45502108335494995
rotation: 1, max_val: 0.16419143974781036
best_rotation: 0
rotation complete
aliment start
make reference image shape: (1566, 2254) reqtangle h,w: 40.170731707317074 40.98245614035088
aliment complete, best match: 0.9390972256660461 angle: 0.1111111111111111 scale: 1.0033333333333334 1.0150000000000001 shape (1566, 2254, 3)


In [5]:
sheet.draw_grid()

make reference image shape: (1566, 2254, 3) reqtangle h,w: 40.15384615384615 40.981818181818184
